In [58]:
import requests
import bs4
import pandas as pd
import nest_asyncio

import asyncio
import aiohttp
# import xml.etree.ElementTree as ET


In [10]:
api_key = '7a4d794242626c6135374b59726852'
# url = f"http://openapi.seoul.go.kr:8088/{api_key}/xml/disabledCalltaxi/1/5/20240302"
url = f"http://openapi.seoul.go.kr:8088/{api_key}/xml/disabledCalltaxi/1/5/"

In [17]:
start_date_2023='20230101'
end_date_2023='20231231'
date_list= [d.strftime('%Y%m%d') for d in pd.date_range(start=start_date_2023, end=end_date_2023, freq='D')]

request_urls = []
for date in date_list:
    request_urls.append(f"{url}{date}")


In [15]:
response = requests.get(url)
content = response.text

### xml을 DataFrame으로 변환하기 ###
#bs4 사용하여 item 태그 분리

xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('item')

# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

#xml값 DataFrame으로 만들기
corona_df = pd.DataFrame(row_list, columns=name_list)


In [71]:
async def get_list(url: str):
    async with aiohttp.ClientSession() as session:
        async with session.get(url ) as response:
            return await response.text()

In [72]:
async def get_xml_data():
    xml_list = [get_list(url) for url in request_urls]
    response_list = await asyncio.gather(*xml_list)
    return response_list

In [73]:
nest_asyncio.apply()
response_list = asyncio.run(get_xml_data())


print(response_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [74]:
print(len(response_list))
def make_df(response_list: list):
    xml_obj = bs4.BeautifulSoup(response_list,'lxml-xml')
    rows = xml_obj.findAll('item')

    # 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
    row_list = [] # 행값
    name_list = [] # 열이름값
    value_list = [] #데이터값
    for i in range(0, len(rows)):
        columns = rows[i].find_all()
        #첫째 행 데이터 수집
        for j in range(0,len(columns)):
            if i ==0:
                # 컬럼 이름 값 저장
                name_list.append(columns[j].name)
            # 컬럼의 각 데이터 값 저장
            value_list.append(columns[j].text)
        # 각 행의 value값 전체 저장
        row_list.append(value_list)
        # 데이터 리스트 값 초기화
        value_list=[]

    #xml값 DataFrame으로 만들기
    return pd.DataFrame(row_list, columns=name_list)

365


In [75]:
dfs = [make_df(li) for li in response_list]
print(dfs)

[       no cartype             receipttime                 settime  \
0    6752    중형승합              2023-01-01  2023-01-01 오전 12:53:06   
1    6752    중형승합              2023-01-01  2023-01-01 오전 12:53:06   
2    5709    중형승합  2023-01-01 오전 12:00:35  2023-01-01 오전 12:34:19   
3    5709    중형승합  2023-01-01 오전 12:01:39   2023-01-01 오전 1:02:43   
4    3711    중형승합  2023-01-01 오전 12:02:12  2023-01-01 오전 12:46:13   
..    ...     ...                     ...                     ...   
244  6164   중형 승합   2023-01-01 오전 9:07:43   2023-01-01 오전 9:27:43   
245  3710    중형승합   2023-01-01 오전 9:07:59   2023-01-01 오전 9:10:08   
246  3624   중형 승합   2023-01-01 오전 9:08:00   2023-01-01 오전 9:24:43   
247  3624   중형 승합   2023-01-01 오전 9:08:00   2023-01-01 오전 9:24:43   
248  8246    중형승합   2023-01-01 오전 9:08:00   2023-01-01 오전 9:09:00   

                   ridetime startpos1 startpos2 endpos1 endpos2  
0     2023-01-01 오전 1:19:19       도봉구     도봉제2동     노원구    상계2동  
1     2023-01-01 오전 1:19:19       도봉구 

In [77]:
new_data = pd.concat(dfs, ignore_index=True)

In [80]:
new_data.to_csv('./call_taxi.csv')